In [1]:
import theano
import theano.tensor as T
import keras


Using gpu device 0: GeForce 940M (CNMeM is disabled, CuDNN not available)


In [ ]:
#print(theano.config)

In [ ]:
#np.distutils.__config__.show()

In [ ]:
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = theano.function([x], s)
logistic([[0, 1], [-1, -2]])

In [ ]:
x = T.scalar('x',dtype='float64')
compute = theano.function([x], 2*x);
compute(10)


### dram-> vram / bottleneck

In [ ]:
x= T.scalar('x',dtype='float64')
compute = theano.function([x], 2*x**2)
compute(2)


### direct to gpu

In [ ]:
x_value = theano.shared(10.0)
compute = theano.function([], 2*x , givens={x:x_value})
compute()

In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.cross_validation import train_test_split
from keras.layers import Convolution2D ,MaxPooling2D,Flatten
from keras.utils import np_utils
import pickle

Using Theano backend.
C:\Users\Back_jud\Anaconda2\lib\site-packages\theano\tensor\signal\downsample.py:6: UserWarning:

downsample module has been moved to the theano.tensor.signal.pool module.



In [ ]:
from keras import backend as K


In [3]:
a = open('d://labels.p')
b = open('d://images.p')
labels = np.array(pickle.load(a))
imgs = np.array(pickle.load(b))

In [4]:
labels = np_utils.to_categorical(labels,nb_classes=2)

In [5]:
x_tr,x_te,y_tr,y_te = train_test_split(imgs,labels,test_size= 0.2,random_state= 123)

In [83]:
model1 = Sequential()
model1.add(Dense(50, input_dim=2500, activation="sigmoid", init='uniform'))
model1.add(Dense(2, activation="softmax", init='uniform'))
model1.compile(loss='categorical_crossentropy', optimizer=SGD())

INFO (theano.gof.compilelock): Refreshing lock C:\Users\Back_jud\AppData\Local\Theano\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64\lock_dir\lock
INFO:theano.gof.compilelock:Refreshing lock C:\Users\Back_jud\AppData\Local\Theano\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64\lock_dir\lock
WARNING (theano.gof.compilelock): Refreshing lock failed, we release the lock before raising again the exception


IOError: [Errno 2] No such file or directory: 'C:\\Users\\Back_jud\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64\\lock_dir\\lock'

In [ ]:
%time hist1 = model1.fit(np.array(x_tr), np.array(y_tr), nb_epoch=300, batch_size=15,show_accuracy=True)

In [ ]:
d= model1.get_weights()[0]

In [ ]:
d1

In [ ]:
plt.imshow(np.reshape(d[:,4],(50,50)),cmap='hot')

In [ ]:
model1.evaluate(np.array(x_te),np.array(y_te),batch_size=10,show_accuracy=True)

In [ ]:
model1.predict_classes(np.array(x_te),2)

In [ ]:
plt.plot(hist1.history['acc'])

In [127]:
#imgs for CNN
imgs2d= []
for img in imgs:
    imgs2d.append(np.reshape(img,(1,50,50)))
    

In [128]:
x_tr2,x_te2,y_tr2,y_te2 = train_test_split(imgs2d,labels,test_size= 0.2,random_state= 123)

In [146]:
model2 = Sequential()
model2.add(Convolution2D(50,5, 5, border_mode='same', input_shape=(1, 50, 50)))
model2.add(Activation('relu'))
model2.add(Convolution2D(50, 5, 5,init='uniform'))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Convolution2D(100, 5, 5, border_mode='same'))
model2.add(Activation('relu'))
model2.add(Convolution2D(100, 5, 5,init='uniform'))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(500,init='uniform'))
model2.add(Activation('relu'))
model2.add(Dense(2,activation='softmax'))


model2.compile(loss='mse', optimizer=SGD(lr=0.01, decay=1e-6, momentum=1.1, nesterov=True))

mod.cu

['nvcc', '-shared', '-O3', '--maxrregcount=32', '--use-local-env', '--cl-version=2013', '-arch=sm_50', '-Xlinker', '/DEBUG', '-D HAVE_ROUND', '-m64', '-Xcompiler', '-DCUDA_NDARRAY_CUH=18715462c72ed6afcd7ca5d52813ce90,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,/Zi,/MD', '-IC:\\Users\\Back_jud\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64\\cuda_ndarray', '-IC:\\Users\\Back_jud\\Anaconda2\\lib\\site-packages\\numpy\\core\\include', '-IC:\\Users\\Back_jud\\Anaconda2\\include', '-IC:\\Users\\Back_jud\\Anaconda2\\lib\\site-packages\\theano\\gof', '-IC:\\Users\\Back_jud\\Anaconda2\\lib\\site-packages\\theano\\sandbox\\cuda', '-o', 'C:\\Users\\Back_jud\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64\\tmpa7_mya\\884cea2a5e11fb3f992663ca999afaa3.pyd', 'mod.cu', '-LC:\\Users\\Back_jud\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.10586-Intel6

1 #include <Python.h>
2 #include <iostream>
3 #include "theano_mod_helper.h"
4 #include "cuda_ndarray.cuh"
5 //////////////////////
6 ////  Support Code
7 //////////////////////
8 
9 
10 #define INTDIV_POW2(a, b) (a >> b)
11 #define INTMOD_POW2(a, b) (a & ((1<<b)-1))
12         // GpuElemwise{Sub}[(0, 0)]
13 // node.op.destroy_map={0: [0]}
14 //    Input   0 CudaNdarrayType(float32, matrix)
15 //    Input   1 CudaNdarrayType(float32, matrix)
16 //    Output  0 CudaNdarrayType(float32, matrix)
17 static __global__ void kernel_Sub_node_884cea2a5e11fb3f992663ca999afaa3_0_1(unsigned int numEls
18 	, const int dim0
19 	, const float * i0_data, int i0_str_0
20 	, const float * i1_data, int i1_str_0
21 	, float * o0_data, int o0_str_0
22 	)
23 {
24     const int idx = blockIdx.x * blockDim.x + threadIdx.x;
25     const int numThreads = blockDim.x * gridDim.x;
26     for (int i = idx; i < numEls; i += numThreads) {
27         int ii = i;
28         const float * ii_i0_data = i0_data;
29       

Exception: ('The following error happened while compiling the node', GpuElemwise{Sub}[(0, 0)](GpuSoftmaxWithBias.0, GpuFromHost.0), '\n', 'nvcc return status', 2, 'for cmd', 'nvcc -shared -O3 --maxrregcount=32 --use-local-env --cl-version=2013 -arch=sm_50 -Xlinker /DEBUG -D HAVE_ROUND -m64 -Xcompiler -DCUDA_NDARRAY_CUH=18715462c72ed6afcd7ca5d52813ce90,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,/Zi,/MD -IC:\\Users\\Back_jud\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64\\cuda_ndarray -IC:\\Users\\Back_jud\\Anaconda2\\lib\\site-packages\\numpy\\core\\include -IC:\\Users\\Back_jud\\Anaconda2\\include -IC:\\Users\\Back_jud\\Anaconda2\\lib\\site-packages\\theano\\gof -IC:\\Users\\Back_jud\\Anaconda2\\lib\\site-packages\\theano\\sandbox\\cuda -o C:\\Users\\Back_jud\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64\\tmpa7_mya\\884cea2a5e11fb3f992663ca999a

In [ ]:
%time hist2 = model2.fit(np.array(x_tr2), y_tr2, nb_epoch=100 , batch_size=36 , show_accuracy=True , shuffle=True)

In [ ]:
hist2.history['acc']

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
!start c:\users\back_jud\anaconda2\lib\site-packages\keras\layers\